In [8]:
import pandas as pd
import os

def find_bug(df):
    """
    df_offers имеет странные дублирующиеся model_id_pred. Векроятно это баг в функции top_n_similar 
    (не находит достаточное количество до n шт матчей + дальнейший convert_columns_to_int)
    """
    columns_to_check = ['model_id_pred_1', 'model_id_pred_2', 'model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
    duplicate_rows_indices = df[df[columns_to_check].apply(lambda row: row.duplicated().sum() >= 1, axis=1)].index
    new_df = df.loc[duplicate_rows_indices]
    return new_df

def merge_columns_to_list(df, n, column_name):
    """
    Функция для объединения значений из нескольких колонок в один список.
    
    Параметры:
        df (DataFrame): Исходный DataFrame.
        n (int): Количество колонок, которые нужно объединить.
        column_name (str): Имя обрабатываемых колонок.
        
    Возвращает:
        DataFrame: DataFrame с добавленным столбцом, содержащим объединенные значения в виде списка.
    """
    # Создаем список колонок для объединения
    columns = [f"{column_name}_{i}" for i in range(1, n+1)]
    
    # Объединяем значения в список
    df[column_name] = df[columns].apply(lambda row: row.tolist(), axis=1)
    
    return df

def exclude_etalon_from_df(df, df_etalon, imya_kategorii):
    df_exclude = df_etalon[df_etalon.category_name == imya_kategorii]
    print(f"{imya_kategorii}: exclude {len(df_exclude)} rows")
    return df[~df.name.isin(df_exclude.name)]

pth_models = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/2801_models_Apple.csv"
pth_offers = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/colbert-2998-finish_offers_top_n_Apple.csv"
# pth_csv_exclude = '/home/sondors/Documents/price/offers_clusterization/data/14_categories_balanced.csv'

id_category = {
        # 3902: 'диктофоны, портативные рекордеры',
        # 510402: 'электронные книги',
        # 4302: 'автомобильные телевизоры, мониторы',
        # 2815: 'смарт-часы и браслеты',
        # 3901: 'портативные медиаплееры',
        # 3904: 'портативная акустика',
        2801: 'мобильные телефоны',
        # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
        # 510401: 'планшетные компьютеры и мини-планшеты',
        # 2102: 'наушники, гарнитуры, наушники c микрофоном',
        # 3903: 'радиоприемники, радиобудильники, радиочасы',
        # 3907: 'магнитолы',
        # 280801: 'GPS-навигаторы'
        }

# id_category = {
    # 921201: 'корм для кошек',
    # 963401: 'колбасы',
    # 710502: 'стиральные машины',
    # 977135: 'ботинки, полуботинки',
    # 911906: 'парфюмерия',
    # 976132: 'туфли и лоферы',
    # 740101: 'кухонные мойки',
    # 7106: 'микроволновые печи',
    # 921401: 'корм для собак',
    # 963302: 'пицца'
    # }

df_models = pd.read_csv(pth_models, sep=";")
# df_models = df_models.drop(columns=['average_price', 'comment'])
df_offers = pd.read_csv(pth_offers, sep=";")
df_offers = df_offers[df_offers.model_id != 0]
# df_offers = df_offers.drop(columns=['true_match', 'false_match'])

# df_exclude = pd.read_csv(pth_csv_exclude, sep=';')
# for category in id_category.values():
#     df_new = exclude_etalon_from_df(df_offers, df_exclude, category)

df_offers = merge_columns_to_list(df_offers, 5, "model_id_pred")
df_offers = merge_columns_to_list(df_offers, 5, "similarity")
df_offers


,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
941,Apple iPhone 15 Pro 512ГБ Blue Titanium (Синий...,7883751,1870824692,True,мобильные телефоны,2801,124310.0,"[{'id': 14522, 'type': 'b', 'value': 1.0}, {'i...",Цвет: Синий титан,Телефоны :: Телефоны Apple iPhone,...,7883751,31.17,7883753,31.16,7883752,31.12,7883743,30.97,"[7883750, 7883751, 7883753, 7883752, 7883743]","[31.25, 31.17, 31.16, 31.12, 30.97]"
942,Смартфон Apple iPhone 15 Plus 512GB Pink (Dual...,7883704,1876207139,False,мобильные телефоны,2801,176990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883697,31.36,7883691,31.34,7883692,31.34,7883695,31.34,"[7883696, 7883697, 7883691, 7883692, 7883695]","[31.36, 31.36, 31.34, 31.34, 31.34]"
943,Смартфон Apple iPhone 15 512GB Yellow для друг...,7883657,1876207142,False,мобильные телефоны,2801,125990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883647,31.38,7883651,31.36,7883652,31.36,7883648,31.31,"[7883646, 7883647, 7883651, 7883652, 7883648]","[31.38, 31.38, 31.36, 31.36, 31.31]"
944,Смартфон Apple iPhone 15 512GB Black (Dual Sim...,7883660,1876207148,False,мобильные телефоны,2801,125990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883646,31.28,7883651,31.28,7883652,31.27,7883648,31.23,"[7883647, 7883646, 7883651, 7883652, 7883648]","[31.3, 31.28, 31.28, 31.27, 31.23]"
945,Смартфон Apple iPhone 14 128GB Желтый для друг...,7883425,1876693234,False,мобильные телефоны,2801,76990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883415,31.52,7883413,31.52,7883412,31.52,7883410,31.52,"[7883416, 7883415, 7883413, 7883412, 7883410]","[31.52, 31.52, 31.52, 31.52, 31.52]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,Смартфон Apple iPhone 15 Pro Max 256Gb eSIM + ...,7883776,1873612511,False,мобильные телефоны,2801,149990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Аппарат поддерживает 1 физическую SIM и eSIM. ...,Смартфоны :: Apple,...,7883766,31.45,7883777,31.44,7883769,31.44,7883768,31.44,"[7883775, 7883766, 7883777, 7883769, 7883768]","[31.47, 31.45, 31.44, 31.44, 31.44]"
8130,Смартфон Apple iPhone 15 Pro Max 256Gb eSIM on...,7883766,1873611822,False,мобильные телефоны,2801,149990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Аппарат не поддерживает физические SIM, только...",Смартфоны :: Apple,...,7883774,31.42,7883776,31.42,7883777,31.42,7883766,31.42,"[7883775, 7883774, 7883776, 7883777, 7883766]","[31.45, 31.42, 31.42, 31.42, 31.42]"
8131,Смартфон Apple iPhone 15 Pro 256Gb eSIM + SIM ...,7883739,1873611850,False,мобильные телефоны,2801,135990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Аппарат поддерживает 1 физическую SIM и eSIM. ...,Смартфоны :: Apple,...,7883730,31.42,7883732,31.42,7883733,31.42,7883736,31.36,"[7883731, 7883730, 7883732, 7883733, 7883736]","[31.42, 31.42, 31.42, 31.42, 31.36]"
8132,"Смартфон Apple iPhone 15 Pro 512 ГБ, Dual eSIM...",7883743,1878760256,True,мобильные телефоны,2801,114990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Цвет: Синий титан | Автофокус: Да | Диагональ ...,Смартфоны :: iPhone :: iPhone 15 Pro,...,7883742,31.53,7883745,31.53,7883744,31.50,7883749,31.44,"[7883743, 7883742, 7883745, 7883744, 7883749]","[31.53, 31.53, 31.53, 31.5, 31.44]"


In [9]:
def accuracy_top_n(offer_model_id, offer_model_id_pred, top_k):
    correct_predictions = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        if true_id in top_k_predictions:
            correct_predictions += 1

    accuracy = correct_predictions / len(offer_model_id)
    return accuracy

def calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k): # проверить функцию
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        
        if true_id in top_k_predictions:
            true_positives += 1
        else:
            false_negatives += 1
            false_positives += (true_id in predicted_ids)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

def calc_acc_for_category(df_offers, df_models, imya_kategorii):
    print(f"{imya_kategorii}")
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_names = list(df_offers['name'])
    offer_model_id = list(df_offers['model_id'])
    offer_model_id_pred = list(df_offers['model_id_pred'])

    model_full_names = list(df_models['full_name'])

    print(f"\tофферов {len(offer_model_id)} шт поиск матча для каждого среди {len(model_full_names)} шт моделей:\n")

    top1_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 1)
    top2_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 2)
    top3_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 3)
    top4_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 4)
    top5_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 5)

    top1_precision, top1_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=1)
    top3_precision, top3_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=3)
    top5_precision, top5_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=5)

    print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}')
    print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}')
    print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}\n')

    # print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}, Precision = {top1_precision:.2%}, Recall = {top1_recall:.2%}')
    # print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}, Precision = {top3_precision:.2%}, Recall = {top3_recall:.2%}')
    # print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}, Precision = {top5_precision:.2%}, Recall = {top5_recall:.2%}\n')
    return (top1_accuracy, top2_accuracy, top3_accuracy, top4_accuracy, top5_accuracy)

for category in id_category.values():
    accuracies_top_n = calc_acc_for_category(df_offers, df_models, category)

мобильные телефоны
	офферов 7193 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 7.66%
	Top-3: Accuracy = 24.69%
	Top-5: Accuracy = 33.16%



In [10]:
def write_errors(df_offers, df, df_models, dst_fld, id_category):
    os.makedirs(dst_fld, exist_ok=True)
    for category in id_category.values():
        category_error = df[df.category_name == category]
        category_all = df_offers[df_offers.category_name == category]
        accuracies_top_n = calc_acc_for_category(category_error, df_models, category)
        with open(os.path.join(dst_fld, f"{category}.txt"), "a") as txt:
            txt.write(f'Всего офферов {len(category_all)} шт из них ошибок {len(category_error)} шт\nРаспределение ошибок:\n')
            txt.write(f'\nTop-1: Accuracy = {accuracies_top_n[0]:.2%}')
            txt.write(f'\nTop-2: Accuracy = {accuracies_top_n[1]:.2%}')
            txt.write(f'\nTop-3: Accuracy = {accuracies_top_n[2]:.2%}')
            txt.write(f'\nTop-4: Accuracy = {accuracies_top_n[3]:.2%}')
            txt.write(f'\nTop-5: Accuracy = {accuracies_top_n[4]:.2%}\n\n')
            txt.write("Истинная модель:\nПоле name оффера --> Найденная модель топ1")
            txt.write("\n____________________________________________________________\n")
            for index, row in category_error.iterrows():
                model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
                model_pred = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
                txt.write(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")                
def exclude_from_df(df, col):
    return df[df['model_id'] != df[col]].reset_index(drop=True)

# clms = ['model_id_pred_1','model_id_pred_2','model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
# df = df_offers.copy()
# for i in range(len(clms)):
#     df = exclude_from_df(df, clms[i])
#     dst_fld = f"/home/sondors/Documents/price/ColBERT/EVAL/errors_ColBERT/exclude_top_{i+1}"
#     write_errors(df_offers, df, df_models, dst_fld, id_category)

мобильные телефоны
	офферов 6642 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 18.44%
	Top-5: Accuracy = 27.61%

мобильные телефоны
	офферов 6093 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 11.09%
	Top-5: Accuracy = 21.09%

мобильные телефоны
	офферов 5417 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 11.24%

мобильные телефоны
	офферов 5217 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 7.84%

мобильные телефоны
	офферов 4808 шт поиск матча для каждого среди 698 шт моделей:

	Top-1: Accuracy = 0.00%
	Top-3: Accuracy = 0.00%
	Top-5: Accuracy = 0.00%



In [11]:
df

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
0,Смартфон Apple iPhone 15 Plus 512GB Pink (Dual...,7883704,1876207139,False,мобильные телефоны,2801,176990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883697,31.36,7883691,31.34,7883692,31.34,7883695,31.34,"[7883696, 7883697, 7883691, 7883692, 7883695]","[31.36, 31.36, 31.34, 31.34, 31.34]"
1,Смартфон Apple iPhone 15 512GB Yellow для друг...,7883657,1876207142,False,мобильные телефоны,2801,125990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883647,31.38,7883651,31.36,7883652,31.36,7883648,31.31,"[7883646, 7883647, 7883651, 7883652, 7883648]","[31.38, 31.38, 31.36, 31.36, 31.31]"
2,Смартфон Apple iPhone 15 512GB Black (Dual Sim...,7883660,1876207148,False,мобильные телефоны,2801,125990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883646,31.28,7883651,31.28,7883652,31.27,7883648,31.23,"[7883647, 7883646, 7883651, 7883652, 7883648]","[31.3, 31.28, 31.28, 31.27, 31.23]"
3,Смартфон Apple iPhone 14 128GB Желтый для друг...,7883425,1876693234,False,мобильные телефоны,2801,76990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883415,31.52,7883413,31.52,7883412,31.52,7883410,31.52,"[7883416, 7883415, 7883413, 7883412, 7883410]","[31.52, 31.52, 31.52, 31.52, 31.52]"
4,Смартфон Apple iPhone 14 128GB Желтый для друг...,7883425,1876705409,False,мобильные телефоны,2801,76990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","2G, 3G, 4G, 5G, Wi-Fi; ОС iOS; Дисплей сенсорн...",Мобильные телефоны,...,7883415,31.52,7883413,31.52,7883412,31.52,7883410,31.52,"[7883416, 7883415, 7883413, 7883412, 7883410]","[31.52, 31.52, 31.52, 31.52, 31.52]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803,Смартфон Apple iPhone 15 Pro 512GB BLUE (MTQG3...,7883751,1875670872,False,мобильные телефоны,2801,170468.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",NaN,Аудио/видео техника :: Мобильные телефоны,...,7883745,31.06,7883742,31.05,7883746,31.05,7883748,31.05,"[7883743, 7883745, 7883742, 7883746, 7883748]","[31.06, 31.06, 31.05, 31.05, 31.05]"
4804,Смартфон Apple iPhone 15 256GB nanoSim+eSim (Р...,7883644,1876117553,False,мобильные телефоны,2801,84970.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",NaN,iPhone 15,...,7883634,31.16,7883632,31.14,7883633,31.14,7883631,31.14,"[7883635, 7883634, 7883632, 7883633, 7883631]","[31.16, 31.16, 31.14, 31.14, 31.14]"
4805,Смартфон Apple iPhone 15 Pro Max 256Gb eSIM + ...,7883776,1873612511,False,мобильные телефоны,2801,149990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Аппарат поддерживает 1 физическую SIM и eSIM. ...,Смартфоны :: Apple,...,7883766,31.45,7883777,31.44,7883769,31.44,7883768,31.44,"[7883775, 7883766, 7883777, 7883769, 7883768]","[31.47, 31.45, 31.44, 31.44, 31.44]"
4806,Смартфон Apple iPhone 15 Pro 256Gb eSIM + SIM ...,7883739,1873611850,False,мобильные телефоны,2801,135990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Аппарат поддерживает 1 физическую SIM и eSIM. ...,Смартфоны :: Apple,...,7883730,31.42,7883732,31.42,7883733,31.42,7883736,31.36,"[7883731, 7883730, 7883732, 7883733, 7883736]","[31.42, 31.42, 31.42, 31.42, 31.36]"


In [12]:
# df.to_csv("/home/sondors/Documents/price/ColBERT_data/10_categories/740101/test/740101_model_id_not_in_top_5.csv", sep=';',index=False)